<a href="https://colab.research.google.com/github/JoungMinJu/lab_study/blob/main/11%EC%9E%A5_RNN%ED%85%8D%EC%8A%A4%ED%8A%B8%EB%B6%84%EB%A5%98/%EB%82%98%EC%9D%B4%EB%B8%8C%EB%B2%A0%EC%9D%B4%EC%A6%88_%EB%B6%84%EB%A5%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.datasets import fetch_20newsgroups
newsdata=fetch_20newsgroups(subset='train')
print(newsdata.keys()) #어떤 속성으로 구성되어있는지 출력

#subset에 all 넣으면 전체 데이터. 위는 train데이터만 가져옴


dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [2]:
#훈련용 샘플의 개수 파악


print (len(newsdata.data), len(newsdata.filenames), len(newsdata.target_names), len(newsdata.target))

#총 11,314개. 그리고 target_names는 이 데이터의 20개의 카테고리의 이름을 담고 있다.


11314 11314 20 11314


In [3]:
#어떤 카테고리들로 구성되어있는가
print(newsdata.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [4]:
#첫번째 샘플은 몇 번 카테고리인가
print(newsdata.target[0])

#해당 카테고리의 제목은 무엇인가
print(newsdata.target_names[7])

7
rec.autos


In [5]:
print(newsdata.data[0])

#글을 보니 스포츠카에 관한 글 같은데 이는 7번 레이블에 속하고 7번은 rec.autos 카테고리를 의미하는구나


From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







나이브 베이즈 분류


In [6]:
#전처리 진행
#토큰화가 전혀 되어있지 않으므로 나이브 베이즈 분류를 위해 데이터를 BoW로 만들어주저야한다.
#나이브 베이즈 분류기에서 토큰화 이전의 단어의 순서는 중요하지 않기 때문에! 오직 빈도수만 고려하면 된다.

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB # 다항분포 나이브 베이즈 모델
from sklearn.metrics import accuracy_score #정확도 계산

In [8]:
dtmvector = CountVectorizer()
X_train_dtm = dtmvector.fit_transform(newsdata.data)
print(X_train_dtm.shape)
#자동으로 BoW로 만드는 CountVectorizer를 통해 자동으로 DTM이 만들어졌다.

#11314개는 훈련용 샘플의 개수이고 DTM 관점에서는 문서의 수가 된다.
#130107은 전체 훈련 데이터에 등장한 단어의 수를 의미

#물론 DTM을 그대로 나이브베이즈 분류기에 사용해도 디지만 TF-IDF로 바꿀거다. 근데 이게 무조건 성능의 개선을 도출하진 않음

(11314, 130107)


In [9]:
tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(X_train_dtm)
print(tfidfv.shape)

(11314, 130107)


In [11]:
mod = MultinomialNB()#나이브 베이즈 모델
mod.fit(tfidfv, newsdata.target)

#모델의 입력으로 TF-IDF 행렬과 11,314개의 훈련 데이터에 대한 레이블이 적혀있는 newsdata.target이 들어간다.
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
#alpha=1.0은 라플라스 스무딩이 적용되엉ㅆ음을 의미

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [12]:
newsdata_test = fetch_20newsgroups(subset='test', shuffle=True) #테스트 데이터 갖고오기
X_test_dtm = dtmvector.transform(newsdata_test.data) #테스트 데이터를 DTM으로 변환
tfidfv_test = tfidf_transformer.transform(X_test_dtm) #DTM을 TF-IDF 행렬로 변환

predicted = mod.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(newsdata_test.target, predicted)) #예측값과 실제값 비교

정확도: 0.7738980350504514
